https://github.com/ftsrg/chc2c
extraction

https://www.reddit.com/r/ProgrammingLanguages/comments/1aigns2/discussing_isabellehol_a_proof_assistant_for/kova7l5/

"This is a good question.

The advantage of higher-order logic over first-order logic is that propositions (and predicates) become first-class objects that you can abstract over in the theory.

Similarly, the advantage of type theory over higher-order logic is that types (and type constructors) become first-class objects that you can abstract over in the theory.

In that sense, if you understand what higher order logic gives you over first order logic, then you can understand what type theory gives you over higher order logic by analogy.

In both cases you can work around these limitations to some extent by using the meta-language as a macro system. For instance, if you only had first-order logic but you wanted to write a lemma that says something about a predicate (say, the strong induction lemma for natural numbers that is generic over some predicate P : nat -> prop), you could write the lemma as a macro that takes as input a predicate represented as a meta-level function that produces a formula, and write the proof of the lemma as a macro that generates a new proof for each new predicate P that the lemma is used on.

Similarly, in HOL you cannot write down code that is generic over a type constructor, e.g. code generic over a monad. Nor can you write down lemmas about such generic code. You can write a meta-level macro that does it, similar to how you can work around the limitations of first-order logic regarding propositions and predicates.

The big problem with this workaround is that now all your code and lemmas are checked at the use site instead of the definition site, similar to how C++ templates work. This also means that you can never write down and prove such a generic theorem; you can only generate and check instances of it.

The other main way to work around these problems is to not work in first order logic, but instead work in some theory built on top of first order logic. That's what ZFC set theory does. You can do something analogous in HOL, by not really working with types at all, but rather work in some sufficiently general un(i)typed universe and always work with predicates/subsets of that."

In [ ]:
# diff

# solver for

Hmmm. Wait. This is a guassian eliminator?




In [68]:
import z3
def solve(eqs, x):
   s = z3.SimpleSolver()
   s.add(eqs)
   return s.solve_for([x])
x,y,z = z3.Reals('x y z')
solve([x + y == z], x)

[]

In [9]:
import z3
x,y,z = z3.Reals('x y z')
s = z3.Solver()
s.add(x + y == z)
print(z3.get_version_string())
print(s.solve_for([x]))
#s.solve_for1(x)


4.15.0
[]


In [9]:
z3.IntVal(3).py_value()

3

In [7]:
z3.get_version_string()

'4.15.0'

In [27]:
import z3
x,y,z,p = z3.Reals('x y z p')
s = z3.SimpleSolver()
s.add(x + y == z)
s.add(p + y + 5 == z)
s.check()
print(s.solve_for([x]))

[(z, p + y + 5, And(y + -1*z + p <= -5, y + -1*z + p >= -5)), (x, z + -1*y, And(x + y + -1*z >= 0, x + y + -1*z <= 0))]


https://github.com/Z3Prover/doc/tree/master/synthesiz3
https://github.com/Z3Prover/doc/commit/690d88ba43a7d706d174da2c48279a5d16810a92

# Proof
z3 proof objects were an ast.
They even had modus ponens?
Hmm.
Look at z3 and cvc5 proof objects

https://ceur-ws.org/Vol-3185/paper9527.pdf simple proof format for SMT
https://ceur-ws.org/Vol-3455/invited1.pdf Challenges in SMT Proof Production and Checking for
Arithmetic Reasoning

In [103]:
from kdrag.all import *
s = smt.Solver()
s.set("clause_proof", True)
f = smt.Function("f", smt.IntSort(), smt.BoolSort())
s.add(smt.ForAll([x], f(x)))
s.add(smt.Not(f(1)))
print(s.check())
p = s.proof()
p.decl()
assert not p.sort() == smt.DeclareSort("Proof")
p.sexpr()

unsat


'(let ((a!1 (forall ((x Int)) (! (f x) :pattern ((f x))))))\n(let ((a!2 (assumption (inst a!1 (not (f 1)) (bind 1) (gen 0))\n                       (or (not a!1) (f 1)))))\n  (proof-trail (assumption assumption a!1)\n               (assumption assumption (not (f 1)))\n               a!2\n               false)))'

In [ ]:
Proof = smt.DeclareSort("Proof")
andI = smt.Function("andI", Proof, Proof, Proof) # to build multicontext
prove = smt.Function("prove", smt.BoolSort(), Proof, Proof)
axiom = smt.Function("axiom", smt.BoolSort(), Proof)

#instan = smt.Function("instan", Proof, Proof)

# both need to be multiarity / parametric
#T = smt.TypeVar("T")
#herb = smt.Function("herb", smt.ArraySort(), Proof)
#instan = smt.Function("instan", Proof, Proof)





# spacer

Can we use spacer for something? Inductive theorem proving?
https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/Fixedpoints
https://github.com/Z3Prover/z3/discussions/5013
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/Dagstuhl2019.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/FSU_2023.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py


In [ ]:
def solver_horn(chc):
    # https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py
    s = z3.SolverFor('HORN')
    s.set('engine', 'spacer')
    s.set('spacer.order_children', 2)
    s.add(chc)
    res = s.check()
    if res == z3.sat:
        return res, s.model()
    elif res == z3.unsat:
        return res, s.proof()
    else:
        return res, None
    
def interpolate(As, A, Bs, B, shared):
    # Uninterpreted predicate Itp over shared symbols
    Itp = z3.Function('Itp', [s.sort() for s in shared] + [z3.BoolSort()])

    # first CHC: A ==> Itp
    left = z3.ForAll([a for a in As], z3.Implies(A, Itp(shared)))
    # second CHC: Itp ==> !B
    right = z3.ForAll([b for b in Bs], z3.Implies(Itp(shared), z3.Not(B)))

    # run CHC solver
    res, answer = solve_horn([left, right])

    if res == z3.sat:
        return answer.eval(Itp(shared))
    else:
        return None





# tableau / meson

For modal logics might be nice
Also smt boosted meson is interesting in and of itself.



# brute check
For anything 32bit or less we ought to just brute check it.

float16 for example. Would give me more confidence.
single variable float32 ops



# Solvers
##  isabelle
Graham was saying why not just download isabelle... Hmm.



```bash
emconfigure ./configure --enable-ho
emmake make AR="/home/philip/Downloads/emsdk/upstream/emscripten/emar rcs"
```

 And had to comment out in Makefile.vars the forced CC and AR values


https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho

bitwuzla
cvc5 
ytices
pysmt

bto2
moxi

implement model checking algorithms?



In [ ]:
import pyboolector as btor

btor.Parse("example.btor")



In [ ]:
import urllib.request
import os
import stat
filename = "eprover-ho"
#urllib.request.urlretrieve("https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho", filename)
st = os.stat(filename)
    # Add execute permissions for user, group, and others
os.chmod(filename, st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

In [77]:
from kdrag.solvers import EProverTHFSolver
s = EProverTHFSolver()
s.add(x == y)
s.check()
s.res.stdout

b'# Preprocessing class: HSSSSMSSSSSNFFN.\n# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)\n# Starting new_ho_10 with 1500s (5) cores\n# Starting ho_unfolding_6 with 300s (1) cores\n# Starting sh4l with 300s (1) cores\n# Starting ehoh_best_nonlift_rwall with 300s (1) cores\n# new_ho_10 with pid 996928 completed with status 9\n# ho_unfolding_6 with pid 996929 completed with status 9\n# sh4l with pid 996930 completed with status 9\n# ehoh_best_nonlift_rwall with pid 996931 completed with status 9\n# Schedule exhausted\n# SZS status GaveUp\n'

In [75]:
import subprocess
subprocess.run(["./eprover-ho", "--help"], check=True)


E 3.2.5-ho "Puttabong Moondrop"

Usage: eprover [options] [files]

Read a set of first-order (or, in the -ho-version, higher-order)
clauses and formulae and try to prove the conjecture (if given)
or show the set unsatisfiable.

Options:

   -h
  --help
    Print a short description of program usage and options.

   -V
  --version
    Print the version number of the prover. Please include this with all bug
    reports (if any).

   -v
  --verbose[=<arg>]
    Verbose comments on the progress of the program. This differs from the
    output level (below) in that technical information is printed to stderr,
    while the output level determines which logical manipulations of the
    clauses are printed to stdout. The short form or the long form without
    the optional argument is equivalent to --verbose=1.

   -o <arg>
  --output-file=<arg>
    Redirect output into the named file.

   -s
  --silent
    Equivalent to --output-level=0.

   -l <arg>
  --output-level=<arg>
    Select an outpu

CompletedProcess(args=['./eprover-ho', '--help'], returncode=0)

# egraph

## Mixins vs subclassing

GaussMixin()
t = GaussTheory()
EGraph(GaussTheory(), SympyTheory(), )

self.theories = []

def find(self, x):
    for t in theories:
        t.find(x)

def union():
    todo = []
    for t in theories:
        todo.extend(t.union())


## Lambda
alpha_norm before using add term.

Recursively carry params down through?
Eta long form.

A normal form?

Do slotted. Add some knd of permutation egraph?


In [ ]:
def eta_long(ctx, t : smt.ExprRef):
    if isinstance(t, smt.QuantifierRef):
        

## Gauss



In [ ]:
class GaussEGraph():
    def __init__(self):
        self.eqs = smt.SimpleSolver() # Or sympy grobner basis.
    def union(self, x, y):
        if isinstance(smt.ArithRef):
            self.eqs

class GrobnerEGraph(EGraph):


## Extract

In [ ]:
def extract(self, t : smt.ExprRef, cost_fun = lambda _: 1):
    best_cost = defaultdict(float("inf"))
    best = {}
    while True:
        done = True
        # Terms are taking the place of enodes.
        for t in self.terms:
            eid = self.find(t.get_id())
            cost = cost_fun(t) + sum([best_cost[self._find(c.get_id())] for c in t.children()]) # cost_fun(t.decl()) ?
            if cost < best_cost[eid]:
                best_cost[eid] = cost
                best = {eid : t}
                done = False
        if done:
            break
    #@functools.cache
    def build_best(t):
        t1 = best[self.find(t.get_id())]
        t1.decl()(*[build_best(c) for c in t1.children()])
    return build_best(t)



False

## Proof

In [64]:
s = smt.Solver()
s.set(unsat_core=True)
a,b,c,d, e = smt.Ints('a b c d e')
id_map = {}
def tassert(a):
    tid = a.get_id()
    id_map[tid] = a
    #s.assert_and_track(a, smt.Bool(str(a)))
    s.assert_and_track(a, smt.Bool(str(tid)))
def core():
    core = s.unsat_core()
    return [id_map[int(str(c))] for c in core]


tassert(a == b)
tassert(b == c)
tassert(e == d)
#s.add(a == b)
#s.add(b == c)
#s.assert_and_track(a == b)
#s.assert_and_track(b == c, b == c)
s.add(a != c)
s.check()
#s.unsat_core()
type(core()[0])

z3.z3.BoolRef

In [ ]:
class EgraphProof(EGraph):
    def __init__(self):
        super().__init__()
        self.reasons = {}
        self.solver.set("unsat_core", True)
    def union(self, e1, e2, reason=None):
        super().union(e1, e2)
        if reason is not None:
            self.reasons[e1 == e2] = reason
    def add(self, expr): # Use this indriectio nin Egrpha, so we can use assert and track
        tid = expr.get_id()
        self.terms[tid] = expr
        self.solver.assert_and_track(expr, smt.BoolRef(str(tid)))
    def core(self):
        core = self.solver.unsat_core()
        return [self.terms[int(str(c))] for c in core]


Return instantiations of rules that make this possible
(vs, rule, lhs = rhs)


return ground rewrite system that'll do it? But then each rule needs to be itself dignified. bleh.

I could also use the trick from the third post of using z3 ematcher and grab the instans

Also it'd be neat to extract those into a reified proof object.


## Context
Egraph does support hypothetical equalities?
But I won't ematch unless I'm in that hypothetical situation

ctxs = []
ctx rebuilding - which ctx subsume, which 
Implies(ctx1, ctx2)
Implies(ctx2, ctx1)  ---> 

contextual rewrite rules.

Brute force contextual egraph

Ok so i can build it.
But then what is it for?
Well, if then else is a good example.

add_term(self, ctx, t):
    if smt.is_ite(t):
        ctx.append(t.arg(0)), t.arg(1)
        ctx.append(smt.Not(t.arg(1))), t.arg(0)



There's a lot of pruning possible in these queries.

In [ ]:
from kdrag.all import *
from kdrag.solvers.egraph import EGraph
def ctx_order(ctxs):
    s = smt.Solver()
    eq = []
    le = []
    for ctx1 in ctxs:
        for ctx2 in ctxs:
            s.push()
            s.add(smt.Not(ctx1 == ctx2))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                eq.append((ctx1,ctx2))
            s.push()
            s.add(smt.Not(smt.Implies(ctx1, ctx2)))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                le.append((ctx2,ctx1))

def subsumes(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(smt.Implies(ctx1, ctx2)))
    return s.check() == smt.unsat
def iff(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(ctx1 == ctx2))
    return s.check() == smt.unsat            


x,y,z = smt.Ints('x y z')

ctx1 = smt.And(x == y, y == z)
ctx2 = x == z
assert subsumes(ctx1, ctx2)
assert not subsumes(ctx2, ctx1)
iff(ctx1, ctx1)

class CtxEGraph(EGraph):
    ctxs : set[int]
    subsume : dict[int, set[int]] # "path" transitive closure

    # there's two versions. One where you have pattern vars in ctx or not. or if contexrt is pattern.
    # suearch over all subsumed contexts.
    # def ematch_ctx(self, vs, pctx, lhs):
    def ematch_ctx(self, vs, lhs):
        res = []
        for ctx in self.ctxs:
            #self.substitute(ctx, vs)
            with self.solver: # takes the push?
                self.solver.add(self.terms[ctx])
                res.extend((ctx, match_) for match_ in self.ematch(vs, lhs))
        return res
    #def ctx_sub():
    def rebuild(self):
        super().rebuild()
        oldctxs = self.ctxs
        for ctx1 in self.ctxs:
            for ctx2 in self.ctxs:
                if ctx2 in self.subsume[ctx1]:
                    continue
                else:
                    self.solver.push()
                    self.solver.add(smt.Not(smt.Implies(ctx1, ctx2)))
                    self.solver.pop()





In [ ]:
def add_term_quant(term):
    t = alpha_norm(term)
    todo = [([], t)]
    while todo:
        ctx, t = todo.pop()
        



# datalog
z3 has a datalog, but whatev. Take snakelog approach https://github.com/philzook58/snakelog

Also homommorphism finder. A la CSP / sqlite blog post

Pretty print into ASP?

spacer?


sqlite dump of binary
sqlite dump of clang ast or treesitter




In [ ]:
from kdrag.all import *
x,y,z = smt.Ints("x y z")
edge = smt.Function("edge", smt.IntSort(), smt.IntSort(), smt.BoolSort())
path = smt.Function("path", smt.IntSort(), smt.IntSort(), smt.BoolSort())
edgepath = rw.rule_of_expr(kd.QForAll([x,y], edge(x,y), path(x,y)))
trans = rw.rule_of_expr(kd.QForAll([x,y,z], edge(x,y), path(y,z), path(x,z)))


def compile_body(vs : list[smt.ExprRef], body : smt.BoolRef) -> str:
    todo = [body]
    env = {}
    froms = []
    wheres = []
    counter = 0
    while todo:
        rel = todo.pop()
        if smt.is_eq(rel):
            raise ValueError("Equality not supported")
        elif smt.is_not(rel):
            raise ValueError("Negation not supported")
        elif smt.is_or(rel):
            raise ValueError("Disjunction not supported")
        elif smt.is_and(rel):
            todo.extend(rel.children())
        elif smt.is_true(rel):
            continue
        elif smt.is_app(rel):
            name = rel.decl().name()
            args = rel.children()
            row_name = name + str(counter)
            counter += 1 
            froms.append(f"{name} AS {row_name}")
            for n, arg in enumerate(args):
                if arg in vs:
                    if arg in env:
                        wheres.append(f"{env[arg]} = {row_name}.x{n}")
                    else:
                        env[arg] = f"{row_name}.x{n}"
                else:
                    wheres.append(f"{row_name}.x{n} = {str(arg)}")
        else:
            raise ValueError(f"Unsupported expression: {rel}")
    return env, froms, wheres

def compile_rule(rule : kd.rewrite.Rule) -> str:
    env, froms, wheres = compile_body(rule.vs, rule.hyp)
    name = rule.conc.decl().name()
    selects = []
    for n,arg in  enumerate(rule.conc.children()):
        if arg in rule.vs:
            if arg in env:
                selects.append(f"{env[arg]} AS x{n}") # maybe select as keyword
            else:
                raise ValueError(f"Variable {arg} not found in body")
        else:
            selects.append(f"{arg} AS x{n}")
    froms = ", ".join(froms)
    wheres = " AND ".join(wheres)
    selects = ", ".join(selects)
    return f"""
    INSERT OR IGNORE INTO {name} SELECT DISTINCT {selects}
    {"FROM " + froms if froms else ""}
    {"WHERE " + wheres if wheres else ""}
    """

print(compile_rule(edgepath))
compile_rule(kd.rewrite.rule_of_expr(edge(1,2)))



    INSERT OR IGNORE INTO path SELECT DISTINCT edge0.x0 AS x0, edge0.x1 AS x1
    FROM edge AS edge0
    
    


'\n    INSERT OR IGNORE INTO edge SELECT DISTINCT 1 AS x0, 2 AS x1\n    \n    \n    '

In [ ]:
import sqlite3
class Datalog():
    def __init__(self):
        self.db = sqlite3.connect(":memory:")
    
    def declare_sig(self, sig : list[smt.FuncDeclRef]):
        for f in sig:
            primkey = "(" + ", ".join([f"x{i}" for i in range(f.arity())]) + ")"
            self.db.execute(f"""CREATE TABLE IF NOT EXISTS {f.name()} 
                            ({', '.join([f'x{i} {f.range().name()}' for i in range(f.arity())])}, 
                             PRIMARY KEY {primkey})""")

    def run(self, rule : kd.rewrite.Rule | smt.BoolRef):
        if isinstance(rule, smt.BoolRef):
            rule = kd.rewrite.rule_of_expr(rule)
        sql = compile_rule(rule)
        self.db.execute(sql)

s = Datalog()
s.declare_db([edge, path])
s.run(edge(1,2))
s.run(edge(2,3))
s.run(edgepath)
s.run(trans)
s.run(trans)
s.db.execute("SELECT * FROM path").fetchall()
        

[(1, 2), (2, 3), (1, 3)]

# compile out of smt2

Look at basic compiler books.
Tiger
Essentials
nanopass

smtcc :)
prescheme

undefined behavior.


We can make a meta layer list
https://pypy.org/posts/2024/07/toy-abstract-interpretation.html
https://pypy.org/posts/2022/07/toy-optimizer.html

In [ ]:
x,y,z = smt.BitVecs("x y z", 64)
getarg = smt.Function("getarg", smt.IntSort(), smt.BitVecSort(64))
copy = smt.Function("copy", smt.BitVecSort(64), smt.BitVecSort(64)) # kd.define("copy", [x], x)
prog = [
    a := x + y,
    b := a + z,
    c := a - b,
]
def address_code(prog):
    seen = set()
    for stmt in prog:
        if all(c in seen for c in stmt.children()):
            seen.add(stmt)
        else:
            return False
    return True
# path assumptions could be interesting.

def esimplify(prog):
    uf = {}
    new_prog = []
    for n,stmt in prog:
        s = smt.Solver()


def register(prog):
    live = set()
    regalloc = {}
    availreg = ["r8", "rsi","rdi","rax"]
    for stmt in reversed(prog):
        availreg.append(regalloc[stmt])
        live.remove(stmt)
        for c in stmt.children():
            if c not in live:
                live.add(c)
                if availreg:
                    regalloc[c] = availreg.pop()
                else:
                    # spill
                    raise Exception("No registers available")

    




[x + y, x + y + z, x + y - (x + y + z)]

In [ ]:
from kdrag.all import *
one = smt.BitVecVal(1, 64)
zero = smt.BitVecVal(0, 64)

one + zero

def to_asm(e : smt.ExprRef):
    if isinstance(e, smt.BitVecNumRef):
        return "mov rdi, %d" % e.as_long()
    elif smt.is_add(e):
        to_asm(e.args[0])
        return [
            f"pop %rdi",
            r"add rdi, %d" % e.args[1].as_long()
        ]
    






1 + 0

# vectorize



In [ ]:
# https://jamesbornholt.com/papers/diospyros-asplos21.pdf
BV16 = smt.BitVecSort(16)
BV8 = smt.BitVecSort(8)

x,y,z = smt.BitVecs("x y z", 16)
vadd = smt.Function('vadd', BV16, BV16, BV16)
vsub = smt.Function('vsub', BV16, BV16, BV16)
vmul = smt.Function('vmul', BV16, BV16, BV16)
vneg = smt.Function('vneg', BV16, BV16)
vmac = smt.Function('vmac', BV16, BV16, BV16, BV16)

vadd(x,y) == smt.Concat(smt.Extract(15, 8, x) + smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) + smt.Extract(7, 0, y))

vsub(x,y) == smt.Concat(smt.Extract(15, 8, x) - smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) - smt.Extract(7, 0, y))
vmul(x,y) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y))
vneg(x) == smt.Concat(-smt.Extract(15, 8, x), -smt.Extract(7, 0, x))
vmac(x,y,z) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y) + smt.Extract(15, 8, z),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y) + smt.Extract(7, 0, z))

smt.prove(smt.Concat(smt.Extract(15, 8, x), smt.Extract(7, 0, x)) == x)

vadd(vmul(x,y),z) == vmac(x,y,z)

# asymptotic

https://github.com/teorth/estimates
https://en.wikipedia.org/wiki/Big_O_notation 
https://www.andrew.cmu.edu/user/avigad/Papers/bigo.pdf  Formalizing O notation in Isabelle/HOL

eqaulity isn't equality (unless it is?)

f(x) = O(g(x)) is really
O(g)[f]

But to do this is to ignore the wisdom of the notation perhaps. because formalists can't fit a notation into their rigid paradgims doesn't mean its bad

I could do custom overloading of == to mean asymptotic.
Didn't I have some idea about O ~ id





# pysmt
I never _really_ gave pysmt a fair shake.
It doesn't support lambda. That is a pretty tough blow



In [31]:
from pysmt.shortcuts import Symbol, And, Not, is_sat

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
is_sat(f)
g = f.substitute({varB: varA})
is_sat(g)

False

# lambda unify


In [ ]:
def unify_huet():
    todo = []
    frozen = [] #flexflex?
    subst = {}
    while True:
        while todo:
            current = todo.pop()
            # do unification.

        for p in frozen:
            if flexflex(p):
                continue
            else:
                frozen.remove(p)
                todo.append(p)
                break
        else:
            return subst, frozen
    
        

    

# GAT / refinement
see types.ipynb also


The property of being a proper telescoping mapping requires an smt solve?

why bother teelscoping? Isabelle pure has no telescoping effect? CVody says that helps build model but did not elaborate
https://argo.matf.bg.ac.rs/events/2009/fatpa2009/slides/Wenzel_PureLogicalReasoning%20in%20Isabelle-Isar.pdf

https://arxiv.org/pdf/1911.00399 hott in pure
https://arxiv.org/pdf/cs/9301105 pualson foundation of generic theorem prover


So a conditional judgement can be seen as a fiber just as much as types can (?)
A /\ B /\ C ===> D

|- goal(p)  as a marker. Always provable goal(p) == True
 
unify(pf1, pf2)

Rather than an equalizer, a pullback. t1 as a substituion mapping anyway?
G1 -> G2
G3 -> G2
Two telescope mappings that go to common context G2. Can I build a pullback?




In [ ]:
from dataclasses import dataclass

@dataclass
class Telescope():
    """ x | A(x), y | B(x,y), ... """
    vs : list[smt.ExprRef]
    preds : list[smt.BoolRef]
    # preds : list[kd.Proof] ? |- ForAll([x], P1), ForAll([x,y], P1 => P2), ...  
    # No but the context should be proof recievining, not proof producing.

    def expr(self, P):
        print(self.vs, self.preds, P)
        return kd.QForAll(self.vs, smt.And(self.preds), P)
    


x,y,z = smt.Reals("x y z")
Telescope([x,y], [x > 0, y > x])

@dataclass
class TeleSubst():
    dom : Telescope
    cod : Telescope
    subst: dict[smt.ExprRef, smt.ExprRef]
    # pf : list[smt.Proof] # same length as dom.preds

    def check(self,by=[]):
        pfs = []
        pfs.extend(by)
        for P in self.dom.preds:
            pfs.append(kd.prove(self.cod.expr(smt.substitute(P, *self.subst.items())), by=pfs))
        return pfs
    def __call__(self, P):
        # P is in context dom
        return smt.substitute(P, *self.subst.items())
    def __matmul__(self, other):
        # substutition forms a category
        assert self.dom == other.cod
        TeleSubst(other.dom, self.cod, {smt.substitute(...)})

# allows subtype weakening.
# if it only allowed syntactic substitutions, then we don't really need the proofs

ctx1 = Telescope([x,z], [x >= 0, z >= x])
ctx2 = Telescope([y], [y >= -1])
TeleSubst(ctx1, ctx2, {x: y + 1, z : y + 10}).check()


# is there a unification procedure for making TeleSubst?
# G1 |- t1 =? t2 -| G2
# result is G1 -> G2 substitution? It's assymmetric?
# doesn't seem like the right shape.



[y] [y >= -1] y + 1 >= 0
[y] [y >= -1] y + 10 >= y + 1


[|- ForAll(y, Implies(And(y >= -1), y + 1 >= 0)),
 |- ForAll(y, Implies(And(y >= -1), y + 10 >= y + 1))]

In [ ]:
from kdrag.all import *

def subst(pf : kd.Proof, vs : list[smt.ExprRef], subst : list[smt.ExprRef]) -> kd.Proof:
    vs1, ab = kd.kernel.herb(smt.ForAll(vs, smt.substitute_vars(pf.thm.body(), *reversed(subst))))
    a = kd.kernel.instan([smt.substitute(t, *zip(vs, vs1)) for t in subst], pf)
    return kd.kernel.modus(ab, a)
x,y,z = smt.Reals("x y z")
p = kd.prove(smt.ForAll([x,z], smt.And(z == z, x == x)))
subst(p, [y, z], [y + 1, z])

def weaken(pf, x):


def unify(pf1, pf2) -> Optional[kd.Proof]:
    # unify the two proofs forall x, p1(x) and forall y, p2(y)
    # return a new proof
    vs1, p1 = kd.utils.open_binder(pf1.thm)
    vs2, p2 = kd.utils.open_binder(pf2.thm)
    vs = vs1 + vs2
    subst = kd.utils.unify(vs1+vs2,p1,p2)
    if subst is not None:
        vs = set(vs) - set(subst.keys()) # maybe keep 'em ordered
        return subst_tac(pf1, vs, )

def build_telescope(pf : kd.Proof, vs, hyps, conc):
    # find a substitution based on unifying conc


        

def chain(pf1, pf2):
    # decompose into rules.
    # unify
    # rebuild






|- ForAll([y, z], And(z == z, y + 1 == y + 1))

In [3]:
import kdrag.reflect as reflect
from kdrag.all import *
x = smt.Bool("x")
@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# verilog extract

Silviu makes a good point.
If you're going to be comparing test case outputs,
No need to 



In [ ]:


x,y,z = smt.Bools("x y z")
a,b,c = smt.BitVecs("a b c", 8)
print(to_verilog("test", [x,y], {z : x & y}))
print(to_verilog("test", [a,b], {z : a & b + 7 != b}))
#(a & b != b).decl().name()

def test():
    with open("/tmp/test.v", "w") as f:
        f.write(to_verilog("test", [x,y], {z : x & y}))
    


// Generated by knuckledragger
module test (
   input x,
   input y,
   output z
);
assign z = (x && y);
endmodule

// Generated by knuckledragger
module test (
   input [7:0] a,
   input [7:0] b,
   output z
);
assign z = ((a & (b + 8'd7)) != b);
endmodule



To say that a verilog file has a property is an axiom schema?

In [ ]:
def sby_axiom(modfile, name, ins, outs):
    


# diff




In [ ]:
from kdrag.all import *

def diff(e, x):
    if e.eq(x):
        return 1
    elif smt.is_value(e):
        return 0
    elif smt.is_add(e):
        return smt.Sum([diff(c,x) for c in e.children()])
    elif e.decl() == real.sin:
        return real.cos(e) * diff(e, x)
    elif e.decl() == real.cos:
        return -real.sin(e) * diff(e, x)
    elif e.decl() == real.exp:
        return real.exp(e) * diff(e, x)
    
    


# knuckelproblems


In [6]:


from kdrag.all import *
# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

"""
add = kd.define("add", [n,m], 
    n.match_(
        (Nat.Zero, m),
        (Nat.Succ(n), Nat.Succ(add(n, m)))
))
add.defn
"""
add.defn


|- ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!6)))

In [9]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# vectors

Bitvector operations

V4 style of 


In [ ]:
BV32 = smt.BitVecSort(32)




# Type resgistry

Some 

add_assoc
add_comm
mul_assoc
mul_comm


COuld reaplce sortdispatch using T.add, T.mul and just have ExprRef.__add__ = lambda x,y: x.sort().add(x,y)



BoolSort
RealSort
StringSort
etc


In [82]:
from kdrag.all import *
T = smt.DeclareSort("T")
T1 = smt.DeclareSort("T")
id(T1) == id(T)
T1.get_id() == T.get_id()

True

In [5]:
from kdrag.all import *
smt.sort_registry
import kdrag.theories.nat as nat
smt.sort_registry
import kdrag.theories.bitvec as bv
smt.sort_registry

{2147483659: Int,
 2147483648: Bool,
 2147483658: Real,
 2147483815: NatI,
 2147483814: Nat,
 2147483704: BitVec(1),
 2147483711: BitVec(8),
 2147483898: Seq(BitVec(1)),
 2147483841: BitVecN}

In [ ]:
# hash cons the types so that we can tag useful stuff on them
types = {}

def DeclareSort(name):
    if name not in types:
        types[name] = type(name, (object,), {})
    return types[name]


# Schematic Vars
Explicitly pulling schematic variables out, may let me refer to them in subproofs (How?)

   G |- A
------------- weaken
     G, v fresh |- A
Well that's annoying.

app(forall x, A, t)




forall x, A  

Maybe I should consider this as A Proof extension rather than proof replacement, likke refinement types


```
Gam |- t   x not in t
-----------
Gam, x |- t 

```


If I merged forall intro and removal


This is just |- forall x, P(x) but with some extra junk.

capture avoiding

https://ncatlab.org/nlab/show/substitution

https://en.wikipedia.org/wiki/Admissible_rule
https://cstheory.stackexchange.com/questions/54600/admissible-rules-in-dependent-type-theory

Gam |- A then sigma Gam |- sigma A


The point was that

prove(t, )

def cprove(vs, p, by=)


a curried form of backward proof tracking.
modus for leaves
comp for moves.
but yeah, split
A /\ B
(A => B => A /\ B)
(A => A \/ B)
(B => A \/ B)

A => B => C

but then aplpying move in context is annoying. Hmm.
And I don't give a shit about anything except quantifier moves so it's wasteful effort.

Maybe the callback method is easier. But it'll have so many expensive calls


def intros():
   




In [ ]:
def modus(ab, a):
    assert smt.is_implies(ab) and ab.arg(0).eq(a)
    assert isinstance(ab, Proof) and isinstance(a, Proof)
    return Proof(ab.arg(1), reasons=[ab, a])

def impl_refl(a):
    return Proof(smt.Implies(a,a))

def comp(ab, bc): # cut sort of
    assert smt.is_implies(ab) and smt.is_implies(bc) and ab.arg(1).eq(bc.arg(0))
    assert isinstance(ab, Proof) and isinstance(bc, Proof)
    return Proof(smt.Implies(ab.arg(0), bc.arg(1)), reasons=[ab, bc])


class Lemma():
    self.backward_proof = impl_refl(goal)
    self.topgoal = goal
    self.curgoal = goal
    

def qed():
    assert self.backwardproof.thm.eq(self.topgoal)
    return self.backwardproof
def fixes():
    herb(self.topgoal)
    self.backwardproof = comp(  , self.backwardproof)(modus(ab,a)))
def prove():
    p = self.prove(curgoal)
    self.backwardproof = modus(self.backwardproof, self.proof)
def intros():
    self.curgoal = self.curgoal.arg(0)
    # goal is unchanged?
    # (a -> b) -> topgoal. uhhh. G -> 
    # 
    # /\ (forall vs, hyps => conc) -> topgoal
    # (hyp1 => conc1) => (hyp2 => conc2) => hyp3 => topgoal
    # goal1 => goal2 => topgoal



Goal
  def to_expr
   

                                                


In [ ]:
def deinstan(pf):
    # get substution if last thing was an instan?
    # Super weird idea.

def cprove(
    vs : list[smt.ExprRef],
    thm: smt.BoolRef,
    by = lambda *args: [] #Callable[smt.ExprRefglist[kd.Proof],
    **kwargs
) -> kd.kernel.Proof:
    """Contextual proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(vs, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)



def indprove(v, thm):
    """Inductive proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(v, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)

def indprove(v : smt.ExprRef, thm, by=None, using=None):
    l = kd.Lemma(smt.ForAll(v, thm))
    v1 = l.fix()
    l.induct(v1, using=using)
    l.auto(by=by)
    return l.qed()

In [ ]:
def subst(vs, p : kd.Proof, subst):
    assert isinstance(p, kd.Proof)
    vs1, body = kd.utils.open_binder_unhygienic(p.fm)
    assert kd.utils.free_in(vs - vs1, body)

# vs

# ok so we can achieve of substutiton by combining herb and instan.
def subst(vs, p : kd.Proof, subst):
    goal = smt.ForAll(vs, smt.substitute(p.fm, *subst.items()))
    l = kd.Lemma(goal)
    vs = l.fixes()
    l.qed(by=p(subst[v] for v in vs))
    #vs, pf = kd.kernel.herb(p)



In [ ]:
class Proof():
    ctx : dict[smt.SortRef, int] # How many de bruijn indices are allowed?
    fm : smt.BoolRef
    

In [ ]:
class Proof():
    vs : list[smt.ExprRef] # ctx
    fm : smt.BoolRef | smt.QuantifierRef
    reasons : list[object]
    def __call__(self, *args, vs=[]):
        smt.substitute(self.fm, *zip(self.vs, args))

def free(vs, fm):
    #fm2 = smt.substitute(fm, *[(v, smt.FreshConst(v.sort())) for v in vs])
    return smt.Lambda(vs, fm).body().eq(fm)
    #return fm2.eq(fm)

def lift(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    return Proof(
        vs = [],
        fm = p.fm,
        reasons = [p]
    )
def lift_forall(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    return Proof(
        vs = vs,
        fm = fm,
        reasons = [p]
    )
def lift_free(ctx, p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    assert all(v in ctx for v in p.vs)
    assert free(ctx, p.fm)
    return Proof(
        vs = ctx,
        fm = p.fm,
        reasons = [p]
    )

"""
  Gam |- forall x, P
---------------
 Gam, x |- P

 This is reversed intro rule. Feels odd.
"""
def open(p : Proof) -> Proof:
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    Proof(p.vs + vs, fm, [p])

def intro(n, p : Proof):
    return Proof(p.vs[n:], smt.ForAll(p.vs[:n], p.fm), [p])

def extend(vs, p : Proof) -> Proof:
    s = smt.Solver()
    s.add(smt.ForAll(p.vs, p.fm))
    s.add(smt.Not(smt.ForAll(p.vs + vs, p.fm)))
    res = s.check()
    if res == smt.unsat:
        return Proof(
            vs = p.vs + vs,
            fm = p.fm,
            reasons = [p]
        )
    else:
        raise Exception("Unknown result from solver")


def subst(p : Proof, ctx : list[smt.ExprRef], subst : dict[smt.ExprRef, smt.ExprRef]):
    assert isinstance(p, Proof)
    assert all(v in p.vs for v in  subst.keys())
    assert free(ctx - p.ctx, p.fm)
    #fm = smt.substitute(p.fm, *[(v, smt.FreshConst(v.sort())) for v in ctx])
    fm = smt.substitute(p.fm, *subst.items())
    # I need to check that ctx2 does not appear in fm. Otherwise I could |- even(zero) ---> zero |- even(zero)
    return Proof(
        vs = ctx,
        fm = fm,
        reasons = [p]
    )


"""
G |- p1    G |- p2 ... 
-------
    G |- p

"""

def prove(ctx, expr, by=None):
    allctx = set(ctx)
    for p in by:
        assert isinstance(p, Proof)
        assert p.ctx == ctx
    s.add(smt.Not(smt.ForAll([ctx], smt.Implies(smt.And(hyp.fm for hyp in by), expr))))
    s.check()




Ok so yes, any forall infecting the lemmas is a problem.
And some things get foralls because its hard an annoying 

qed should 


In [6]:
from kdrag.all import *
import kdrag.theories.real as real
x,y = smt.Reals("x y")
cos = real.cos
sin = real.sin
#os_diff = kd.prove(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)), [real.cos_add, real.cos_neg, real.sin_neg])

_l = kd.Lemma(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)))
_x, _y = _l.fixes()
_l.symm()
_l.eq(cos(_x + (-_y)))
#_l.rw(kd.prove(cos(_x - _y) == cos(_x + (-_y))))
#_l.rw(kd.prove(smt.ForAll([x,y], x - y == (x + (-y))))(_x, _y))
_l.rw(real.cos_add(_x, -_y))
#_l.auto(by=[real.cos_add, real.sin_add])
_l.auto(by=[real.cos_neg(_y), real.sin_neg(_y)])
_l.lemmas
_l.qed()
cos_diff = _l.lemmas


# Homomorphism theorems


In [ ]:
from kdrag.all import *
import kdrag.property as prop
import kdrag.theories.algebra.group as group

A = smt.DeclareSort('A')
def Group(A):
    mul = smt.Function('mul', A, A, A)
    inv = smt.Function('inv', A, A)
    id_ = smt.Const('id', A)
    x,y,z = smt.Consts('x y z', A)
    kd.notation.mul.register(A, mul)
    kd.notation.invert.register(A, inv)
    assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
    id_left = kd.axiom(smt.ForAll([x], id_ * x == x))
    inv_left = kd.axiom(smt.ForAll([x], inv(x) * x == id_))
    return id_, assoc, id_left, inv_left


B = smt.DeclareSort('B')
group.Group.register(A)
idA, assocA, id_leftA, inv_leftA = Group(A)
idB, assocB, id_leftB, inv_leftB = Group(B)

h = smt.Function('h', A, B)
def homo(id_, h):
    A = h.domain(0)
    B = h.range()
    x,y,z = smt.Consts("x y z", A)
    mul = smt.ForAll([x,y], h(x * y) == h(x) * h(y))
    id_ = h(idA) == idB
    inv = smt.ForAll([x], h(~x) == ~h(x))
    return mul, id_, inv
homo(h)





ValueError: ('Not registered in typeclass', (A,))

Universe of groups.

I was saying construct it as permuations  Int -> Int

Nontrivial
But we could also driectly axiomatize

* is composition
id is  x -> x
homomorphisms need to... be careful here. Only push homo through on particular Group?
SetGrp 





In [ ]:

Grp = smt.DeclareSort("Grp")
SetGrp = set_.SetSort(Grp)
inv = smt.Function("inv", Grp, Grp)
# inv = smt.Function("inv", GrpSet, GrpSet)
mul = smt.Function("mul", Grp, Grp) # We just leave mul un

Closed = smt.QForAll([x,y, S], S[x], S[y], S[x * y], S[inv(x)])

smt.QForAll([S], Closed[S], S[x], S[y], S[z],  x*(y*z) == (x * y) * z) # Is there a model where mul can be associative and always defined? 
smt.QForAll([S], Closed[S], S[x], inv(x) * x == x)  # maybe unconditionally actually
smt.QForAll([S], id_ * x == x) # also unconditiojnality